In [1]:
import pandas as pd

In [2]:
data_orders = pd.read_csv('cleaned_data.csv')

In [3]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
#Change column type
data_orders['Order_Date'] = pd.to_datetime(data_orders['Order_Date'])

In [5]:
#Show first 5 lines
data_orders.head()

,ID_Order,ID_Customer,ID_Item,Order_Date,Amount_Gross_Order,city_name_fa,Quantity_item,Order_Date_Shamsi,Year,Month,day_of_week,Hour,TotalGrossValue
0,1000740,609924,6906,2013-09-27 12:03:00,812453.00,قم,1.00,1392-07-05,1392,7,Friday,12,812453.00
1,1000411,720568,8777,2013-09-27 19:43:00,3254717.00,تهران,1.00,1392-07-05,1392,7,Friday,19,3254717.00
2,1000977,695557,3136,2013-09-27 22:26:00,8764151.00,شهرکرد,1.00,1392-07-05,1392,7,Friday,22,8764151.00
3,1000919,470250,1532,2013-09-28 12:04:00,926226.00,زنجان,2.00,1392-07-06,1392,7,Saturday,12,1852452.00
4,1002123,531450,8045,2013-09-29 22:08:00,9433585.00,رشت,1.00,1392-07-07,1392,7,Sunday,22,9433585.00


In [6]:
#Last day of available data
reference_date = data_orders['Order_Date'].max() + pd.Timedelta(days=1)
#To analyze RFM, get the number of days since the customer's last purchase, the number of customer purchases, and the customer's total spend.
rfm = data_orders.groupby('ID_Customer').agg({
    'Order_Date': lambda x: (reference_date - x.max()).days,
    'ID_Order': 'count',
    'TotalGrossValue': 'sum'                          
}).rename(columns={
    'Order_Date': 'Recency',
    'ID_Order': 'Frequency',
    'TotalGrossValue': 'Monetary'
})

rfm

,Recency,Frequency,Monetary
ID_Customer,,,
466132,9,11,35449635.00
466136,445,1,306963.00
466141,638,3,1324312.00
466146,1024,1,302752.00
466152,99,3,1173027.00
...,...,...,...
7281740,191,1,340000.00
7281800,187,1,414000.00
7281906,194,1,480000.00


In [7]:
#Get the number of years the customer has been a member
col_year = data_orders.groupby('ID_Customer').agg({
    'Order_Date': lambda x: ((reference_date - x.min()).days) / 365,                      
}).rename(columns={'Order_Date': 'Year'})

col_year.head()

,Year
ID_Customer,
466132,4.95
466136,1.22
466141,2.73
466146,2.81
466152,3.44


In [8]:
# Adding a Membership Year column to the RFM table
rfm = pd.concat([rfm, col_year['Year']], axis=1)

In [9]:
# Setting the number of years of membership to 1 for people with less than 1 year of experience to avoid calculation errors
rfm.loc[rfm['Year'] < 1, 'Year'] = 1

In [10]:
# Obtain the average number of purchases and total purchase price per year for each customer.
rfm['Frequency'] = rfm['Frequency'] / rfm['Year']
rfm['Monetary'] = rfm['Monetary'] / rfm['Year']

In [11]:
# Customers with less recent are better → higher score
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Customers who buy more are better → higher score
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Customers with higher purchase amounts are better → higher score
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm.head()

,Recency,Frequency,Monetary,Year,R_Score,F_Score,M_Score
ID_Customer,,,,,,,
466132,9,2.22,7160551.62,4.95,5,5,5
466136,445,0.82,251778.64,1.22,3,3,3
466141,638,1.10,484828.37,2.73,2,5,3
466146,1024,0.36,107914.53,2.81,1,1,2
466152,99,0.87,341431.30,3.44,5,3,3


In [12]:
# Putting recency, frequency, and monetary value in one column for sorting
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

rfm.head()

,Recency,Frequency,Monetary,Year,R_Score,F_Score,M_Score,RFM_Score
ID_Customer,,,,,,,,
466132,9,2.22,7160551.62,4.95,5,5,5,555
466136,445,0.82,251778.64,1.22,3,3,3,333
466141,638,1.10,484828.37,2.73,2,5,3,253
466146,1024,0.36,107914.53,2.81,1,1,2,112
466152,99,0.87,341431.30,3.44,5,3,3,533


In [13]:
# Create a dictionary to identify the category of each RFM score.
rfm_segments = {}

def category(F, M):
    if F in [1,2] and M in [1,2]:
        return "Low"
    if F == 3 or M == 3:
        return "Medium"
    if F in [4,5] and M in [4,5]:
        return "High"
    return "Medium"  # default

for R in range(1,6):
    for F in range(1,6):
        for M in range(1,6):
            
            FM = category(F,M)

            if R == 5:
                if FM == "Low": seg = "Promising"
                elif FM == "Medium": seg = "Loyal"
                else: seg = "Champions"

            elif R == 4:
                if FM == "Low": seg = "Potentials"
                elif FM == "Medium": seg = "Active Loyal"
                else: seg = "High-Value Active"

            elif R == 3:
                if FM == "Low": seg = "About To Sleep"
                elif FM == "Medium": seg = "Need Attention"
                else: seg = "Valuable Cooling"

            elif R == 2:
                if FM == "Low": seg = "Hibernating"
                elif FM == "Medium": seg = "At Risk"
                else: seg = "Can't Lose Them"

            elif R == 1:
                if FM == "Low": seg = "Lost Low-Value"
                elif FM == "Medium": seg = "Lost Medium-Value"
                else: seg = "Lost High-Value"

            rfm_segments[f"{R}{F}{M}"] = seg


In [14]:
print(rfm_segments)

{'111': 'Lost Low-Value', '112': 'Lost Low-Value', '113': 'Lost Medium-Value', '114': 'Lost Medium-Value', '115': 'Lost Medium-Value', '121': 'Lost Low-Value', '122': 'Lost Low-Value', '123': 'Lost Medium-Value', '124': 'Lost Medium-Value', '125': 'Lost Medium-Value', '131': 'Lost Medium-Value', '132': 'Lost Medium-Value', '133': 'Lost Medium-Value', '134': 'Lost Medium-Value', '135': 'Lost Medium-Value', '141': 'Lost Medium-Value', '142': 'Lost Medium-Value', '143': 'Lost Medium-Value', '144': 'Lost High-Value', '145': 'Lost High-Value', '151': 'Lost Medium-Value', '152': 'Lost Medium-Value', '153': 'Lost Medium-Value', '154': 'Lost High-Value', '155': 'Lost High-Value', '211': 'Hibernating', '212': 'Hibernating', '213': 'At Risk', '214': 'At Risk', '215': 'At Risk', '221': 'Hibernating', '222': 'Hibernating', '223': 'At Risk', '224': 'At Risk', '225': 'At Risk', '231': 'At Risk', '232': 'At Risk', '233': 'At Risk', '234': 'At Risk', '235': 'At Risk', '241': 'At Risk', '242': 'At Risk

In [15]:
# Categorizing and naming each score
rfm['Segment'] = rfm['RFM_Score'].map(rfm_segments)

# If a value is not found, label it Unknown
rfm['Segment'] = rfm['Segment'].fillna('Unknown')

rfm.head(10)

,Recency,Frequency,Monetary,Year,R_Score,F_Score,M_Score,RFM_Score,Segment
ID_Customer,,,,,,,,,
466132,9,2.22,7160551.62,4.95,5,5,5,555,Champions
466136,445,0.82,251778.64,1.22,3,3,3,333,Need Attention
466141,638,1.10,484828.37,2.73,2,5,3,253,At Risk
466146,1024,0.36,107914.53,2.81,1,1,2,112,Lost Low-Value
466152,99,0.87,341431.30,3.44,5,3,3,533,Loyal
466154,3,1.00,28990826.00,1.00,5,3,5,535,Loyal
466163,145,0.42,420985.38,4.74,5,1,3,513,Loyal
466165,35,1.00,12900000.00,1.00,5,3,5,535,Loyal
466204,927,0.74,2059963.52,4.06,1,2,5,125,Lost Medium-Value


In [16]:
rfm['Segment'].value_counts()

Segment
Need Attention       24255
Active Loyal         18073
Loyal                17566
Lost Low-Value       17145
At Risk              16315
Hibernating          13213
Lost Medium-Value    13003
Champions            12535
High-Value Active    12038
Valuable Cooling      4773
About To Sleep        1229
Can't Lose Them        844
Potentials             223
Promising              205
Lost High-Value         65
Name: count, dtype: int64

In [17]:
rfm.reset_index(inplace=True)

In [18]:
rfm.to_csv('data_rfm.csv', index=False, encoding='utf-8-sig')